In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "bsvkvKtIeOMa1amMu5ZlNLCEpRMHPSeb"
mp_id = "mp-505297"

In [3]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# Symétries de la molécule
sym = SpacegroupAnalyzer(prim_struc).get_symmetry_operations()

# Symétries choisies pour les atomes
# Plan miroir (z,y,x)
S1 = sym[7]
# Axe 4-barre | centre d'inversion en (0,0,0)  (-x-y-z,x,y)
S2 = sym[3]
# Axe de rotation C2 (y,x,-x-y-z)
S3 = sym[6]

# Atomes choisis
Nb = [0.0, 0.0, 0.0]
Sb = [0.0, 0.0, 0.5]
Ru = [0.25, 0.25, 0.75]

# Application des symétries
Nb_sym = S1.operate(Nb)
Sb_sym = S2.operate(Sb)
Ru_sym = S3.operate(Ru)

Première symétrie sur l'atome de Nb

In [5]:
print("Opération de symétrie 1: \n")
print("L'atome de départ est le Nb et a comme coordonnées: {}\n".format(Nb))
print("On lui applique une symétrie de plan miroir (z,y,x) :\n")
print(S1.rotation_matrix, "*", Nb, '+', S1.translation_vector)
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Nb_sym))
v1 = quick_view(prim_struc)
display(v1)
v1.script("draw symop 8 {atomno=1}")

Opération de symétrie 1: 

L'atome de départ est le Nb et a comme coordonnées: [0.0, 0.0, 0.0]

On lui applique une symétrie de plan miroir (z,y,x) :

[[-1. -1. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]] * [0.0, 0.0, 0.0] + [0. 0. 0.]
L'atome d'arrivé a comme coordonnées : [0. 0. 0.]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

Deuxième symétrie sur l'atome de Sb

In [6]:
print("Opération de symétrie 2: \n")
print("L'atome de départ est le Sb et a comme coordonnées: {}\n".format(Sb))
print("On lui applique une roto-inversion (-x-y-z,x,y) :")
print(S2.rotation_matrix, "*", Sb, '+', S2.translation_vector)
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Sb_sym))
v1 = quick_view(prim_struc)
display(v1)
v1.script("draw symop 4 {atomno=7}")

Opération de symétrie 2: 

L'atome de départ est le Sb et a comme coordonnées: [0.0, 0.0, 0.5]

On lui applique une roto-inversion (-x-y-z,x,y) :
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -1. -1.]] * [0.0, 0.0, 0.5] + [0. 0. 0.]
L'atome d'arrivé a comme coordonnées : [ 0.   0.5 -0.5]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

Troisième symétrie sur l'atome de Ru

In [7]:
print("Opération de symétrie 3: \n")
print("L'atome de départ est le Ru et a comme coordonnées: {}\n".format(Ru))
print("On lui applique une symétrie d'axe 2 (y,x,-x-y-z) : ")
print(S3.rotation_matrix, "*", Ru, '+', S3.translation_vector)
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Ru_sym))
v1 = quick_view(prim_struc)
display(v1)
v1.script("draw symop 7 {atomno=3}")

Opération de symétrie 3: 

L'atome de départ est le Ru et a comme coordonnées: [0.25, 0.25, 0.75]

On lui applique une symétrie d'axe 2 (y,x,-x-y-z) : 
[[-1. -1. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]] * [0.25, 0.25, 0.75] + [0. 0. 0.]
L'atome d'arrivé a comme coordonnées : [-1.25  0.25  0.25]



JsmolView(layout=Layout(align_self='stretch', height='400px'))